# youtube (댕쪽이 상담소) 크롤링 코드

유투브 링크 txt로 저장 -> txt에서 링크 하나씩 제목,이미지,자막 수집

## 0. 라이브러리 임포트

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

## 1. 유투브 url 수집

In [ ]:
# 1. 크롬 옵션 (창 없이 실행 가능)
options = Options()
options.add_argument('--headless')  # 창 안 띄우고 실행하려면 주석 해제
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# 2. 드라이버 경로
# driver_path = 'chromedriver.exe'
# service = Service(executable_path=driver_path)
# driver = webdriver.Chrome(service=service, options=options)
driver = webdriver.Chrome()
# 3. 유튜브 채널 URL 접속
channel_url = "https://www.youtube.com/@%EB%8C%95%EC%AA%BD%EC%9D%B4%EC%83%81%EB%8B%B4%EC%86%8C/videos"
driver.get(channel_url)
time.sleep(3)

# 4. 스크롤 내려서 더 많은 영상 로드 (선택)
SCROLL_PAUSE_TIME = 2
last_height = driver.execute_script("return document.documentElement.scrollHeight")

for _ in range(5):  # 5번 정도 스크롤 내려서 영상 추가 로드
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# 5. a 태그에서 video-title-link 수집
video_elements = driver.find_elements(By.ID, "video-title-link")
print(f"✅ 영상 수: {len(video_elements)}")

video_urls = []

for elem in video_elements:
    href = elem.get_attribute("href")
    title = elem.text.strip()
    if href:
        video_urls.append(href)
        print(f"🎬 {title}")
        print(f"🔗 {href}")

# 6. 저장
with open("all_video_urls.txt", "w", encoding="utf-8") as f:
    for url in video_urls:
        f.write(url + "\n")

print("\n✅ 수집 완료! 'all_video_urls.txt'에 저장됨.")

driver.quit()


✅ 영상 수: 67
🎬 “이사 후 분리불안, 2주간 출근도 못한 보호자의 이야기” | 강아지 문제 행동교정 | 댕쪽이상담소
🔗 https://www.youtube.com/watch?v=xEPaTtcysoM
🎬 “외출이 불가능해요…” 분리불안에 사람까지 무는 파양견 이야기 | 강아지 문제 행동교정 | 댕쪽이상담소
🔗 https://www.youtube.com/watch?v=5UJ9dULd2Tc
🎬 [고민상담소]”잠도 못자고 너무 힘들어요..”몸이 불편한 구독자님의 절박한 요청에 직접 해결하러 나섰습니다
🔗 https://www.youtube.com/watch?v=MDLfhtfHEXw
🎬 퇴근 후 오랜만에 삼겹살~🥓의 하이라이트(0:00 ~ 4:40)
🔗 https://www.youtube.com/watch?v=LTCNHuiK_vQ
🎬 "엄마랑 산책 안 간다고!!" 끝까지 버티는 “줄다리기왕” 36kg 풍산 믹스 | 강아지 문제 행동교정 | 댕쪽이상담소
🔗 https://www.youtube.com/watch?v=2wE460rf2Bk
🎬 산책 시 다른 강아지만 보면 폭주하는 “폭주왕” 페키니즈 | 강아지 문제 행동교정 | 댕쪽이상담소
🔗 https://www.youtube.com/watch?v=cRKgJBNfMMY
🎬 유기견 보호소에서 안락사 직전 입양 됐지만 사람이 무서워 변까지 지리는 “찌질왕” 믹스견 | 강아지 문제 행동교정 | 댕쪽이상담소
🔗 https://www.youtube.com/watch?v=qD9GqhVFITU
🎬 사람 아이처럼 키워져서 결국 독립을 못한 "불안왕" 푸들 | 강아지 문제 행동교정 | 댕쪽이상담소
🔗 https://www.youtube.com/watch?v=4rgd2m3O1MU
🎬 짖음으로 인한 층간소음으로 민원을 받게 만든 “고음왕” 셔틀랜드 쉽독 | 강아지 문제 행동교정 | 댕쪽이상담소
🔗 https://www.youtube.com/watch?v=gk3w9z2i588
🎬 평생 훈육 없이 자라 제멋대로인 “알빠왕” 믹스 | 강아지 

In [1]:
!pip install youtube-transcript-api
!pip install yt-dlp

   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 20.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------------------------- 3.2/3.2 MB 23.8 MB/s eta 0:00:00


## 2. csv 파일 생성후 [제목, 내용, 주소, 이미지] 저장

- 접근 불가 영상은 패스

In [8]:
import yt_dlp
import csv
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptFound

# 링크 파일 읽기
with open('2_all_video_urls.txt', 'r', encoding='utf-8') as f:
    video_urls = [line.strip() for line in f if line.strip()]

# 결과 저장할 CSV 파일
with open('2_youtube_data.csv', 'w', newline='', encoding='utf-8-sig') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['title', 'content', 'URL', 'image'])  # 헤더

    for url in video_urls:
        try:
            # 영상 ID 추출
            video_id = url.split('v=')[-1].split('&')[0]

            # yt-dlp로 제목, 썸네일 가져오기
            ydl_opts = {
                'skip_download': True,
                'quiet': True,
                'forcejson': True,
            }

            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                info = ydl.extract_info(url, download=False)

            title = info.get('title', '제목 없음')
            thumbnail = info.get('thumbnail', f"https://img.youtube.com/vi/{video_id}/maxresdefault.jpg")

            # 자막 가져오기
            try:
                transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['ko', 'en'])
                captions = '\n'.join([item['text'] for item in transcript])
            except (TranscriptsDisabled, NoTranscriptFound):
                captions = '자막 없음'

            # CSV 저장
            writer.writerow([title, captions, url, thumbnail])

            print(f"✅ {title} 저장 완료!")

        except Exception as e:
            print(f"⚠️ {url} 처리 실패: {e}")

print("📄 모든 유튜브 데이터 저장 완료!")


✅ “이사 후 분리불안, 2주간 출근도 못한 보호자의 이야기” | 강아지 문제 행동교정 | 댕쪽이상담소 저장 완료!
✅ “외출이 불가능해요…” 분리불안에 사람까지 무는 파양견 이야기 | 강아지 문제 행동교정 | 댕쪽이상담소 저장 완료!
✅ [고민상담소]”잠도 못자고 너무 힘들어요..”몸이 불편한 구독자님의 절박한 요청에 직접 해결하러 나섰습니다 저장 완료!


ERROR: [youtube] LTCNHuiK_vQ: Join this channel to get access to members-only content like this video, and other exclusive perks.


⚠️ https://www.youtube.com/watch?v=LTCNHuiK_vQ 처리 실패: ERROR: [youtube] LTCNHuiK_vQ: Join this channel to get access to members-only content like this video, and other exclusive perks.
✅ "엄마랑 산책 안 간다고!!" 끝까지 버티는 “줄다리기왕” 36kg 풍산 믹스 | 강아지 문제 행동교정 | 댕쪽이상담소 저장 완료!
✅ 산책 시 다른 강아지만 보면 폭주하는 “폭주왕” 페키니즈 | 강아지 문제 행동교정 | 댕쪽이상담소 저장 완료!
✅ 유기견 보호소에서 안락사 직전 입양 됐지만 사람이 무서워 변까지 지리는 “찌질왕” 믹스견 | 강아지 문제 행동교정 | 댕쪽이상담소 저장 완료!
✅ 사람 아이처럼 키워져서 결국 독립을 못한 "불안왕" 푸들 | 강아지 문제 행동교정 | 댕쪽이상담소 저장 완료!
✅ 짖음으로 인한 층간소음으로 민원을 받게 만든 “고음왕” 셔틀랜드 쉽독 | 강아지 문제 행동교정 | 댕쪽이상담소 저장 완료!
✅ 평생 훈육 없이 자라 제멋대로인 “알빠왕” 믹스 | 강아지 문제 행동교정 | 댕쪽이상담소 저장 완료!
✅ 저에 대한 기사와 논란에 대해 솔직하게 이야기 해보겠습니다 저장 완료!
✅ 사람처럼 안아주고 달래줘서 버릇이 나빠진 “생떼왕” 포메라니안 | 강아지 문제 행동교정 | 댕쪽이상담소 저장 완료!
✅ 가게에 손님오면 끊임없이 짖는 “관종왕” 포메라니안 | 강아지 문제 행동교정 | 댕쪽이상담소 저장 완료!
✅ 성질 사납고 맘에 안들면 물어버리는 “승질왕” 말티즈 | 강아지 문제 행동교정 | 댕쪽이상담소 저장 완료!


ERROR: [youtube] FH8ET07BJR8: This video is available to this channel's members on level: 댕밀리 (or any higher level). Join this channel to get access to members-only content and other exclusive perks.


⚠️ https://www.youtube.com/watch?v=FH8ET07BJR8 처리 실패: ERROR: [youtube] FH8ET07BJR8: This video is available to this channel's members on level: 댕밀리 (or any higher level). Join this channel to get access to members-only content and other exclusive perks.
✅ 집에서 시도때도 없이 짖는 “상전왕” 말티즈 | 강아지 문제 행동교정 | 댕쪽이상담소 저장 완료!
✅ 옷 벗을 때 몸부림 치는 “난리왕” 푸들 | 강아지 문제 행동교정 | 댕쪽이상담소 저장 완료!
✅ 혼자 집에 있기 싫다고 주인을 집에 꽁꽁 묶어버린 “인질왕” 푸들 | 강아지 문제 행동교정 | 댕쪽이상담소 저장 완료!
✅ 움직이는 모든 것에 반응하고 돌진하는 ”고추왕“ 미니핀 | 강아지 문제 행동교정 | 댕쪽이상담소 저장 완료!
✅ 통제하려고 하면 가족도 물어버리는 “입뽕왕” 진도믹스 | 강아지 문제 행동교정 | 댕쪽이상담소 저장 완료!
✅ 8년동안 마음대로 짖고 마음대로 물면서 살아온 ”어쩔왕“ 폼피츠 | 강아지 문제 행동교정 | 댕쪽이상담소 저장 완료!
✅ 엄마가 마음을 몰라줘서 스스로 무섭게 돌변해버린 “헐크왕” 진도믹스 | 강아지 문제 행동교정 | 댕쪽이상담소 저장 완료!
✅ [AS 왔습니다] 피터지게 싸우던 역대급 “싸움왕” 페키니즈 다시 참교육하러 갔습니다 | 강아지 문제 행동교정 | 댕쪽이상담소 저장 완료!
✅ [무료 솔루션] 보호자님을 제외한 다른 사람에게 부정적이고 경계심이 심한 “애증왕” 푸들 최종편 | 강아지 문제 행동교정 | 댕쪽이상담소 저장 완료!
✅ [무료 솔루션] 보호자님을 제외한 다른 사람에게 부정적이고 경계심이 심한 “애증왕” 푸들 1편 | 강아지 문제 행동교정 | 댕쪽이상담소 저장 완료!
✅ 악.플.읽.기 1탄 | 어둠의 개통령 저장 완료!
✅ [무료 솔루션] 보호자님의 과잉보호

...

⚠️ https://www.youtube.com/watch?v=0LppaLvQ1x8 처리 실패: ERROR: [youtube] 0LppaLvQ1x8: Join this channel to get access to members-only content like this video, and other exclusive perks.
✅ 보호자님의 손과 다리를 물어뜯어야 직성이 풀리는 “엄살왕” 시바견 | 강아지 문제 행동교정 | 댕쪽이상담소 저장 완료!
⚠️ https://www.youtube.com/watch?v=-ZlR0fGs6RQ 처리 실패: no element found: line 1, column 0
✅ 집에 사람이 오면 미친듯이 짖는 "짖음왕" 말티푸 | 강아지 문제 행동교정 | 댕쪽이상담소 저장 완료!
✅ 키워주신 은혜도 모르고 웬수 물어뜯듯이 가족을 공격하는 "폭력왕" 말티푸 | 강아지 문제 행동교정 | 댕쪽이상담소 저장 완료!
✅ 인생 처음으로 보육원과 유기견 보호소에 200만원 기부했습니다 feat : 실버버튼 언박싱 저장 완료!
✅ 안고 있을때 누가 만지려고 하면 물어버리는 “짜증왕” 포메라니안 | 강아지 문제 행동교정 | 댕쪽이상담소 저장 완료!
✅ 친구에게 악령마냥 집착하는 “집착왕“ 프렌치 불독 | 강아지 문제 행동교정 | 댕쪽이상담소 저장 완료!
✅ 목욕할 때 손만 대면 물어버리는 “입질왕” 말티푸 | 강아지 문제 행동교정 | 댕쪽이상담소 저장 완료!
✅ 가게에 상주하는 “요구왕” 말티즈 최종화 | 강아지 문제 행동교정 | 댕쪽이상담소 저장 완료!
✅ 가게에 상주하는 “요구왕” 말티즈 문제 행동교정 1화 | 강아지 문제 행동교정 | 댕쪽이상담소 저장 완료!
✅ 강아지 훈련사의 모닝루틴 국내 최초공개 // 댕쪽이상담소 저장 완료!
✅ 10만 구독자 Q&A하는 현시점 가장 핫한 훈련사 (어둠의 개통령) // 댕쪽이상담소 저장 완료!
⚠️ https://www.youtube.com/watch?v=D9xe9cZetQE 처리 실패: no eleme

ERROR: [youtube] LdEiNPNUCe8: Join this channel to get access to members-only content like this video, and other exclusive perks.


⚠️ https://www.youtube.com/watch?v=LdEiNPNUCe8 처리 실패: ERROR: [youtube] LdEiNPNUCe8: Join this channel to get access to members-only content like this video, and other exclusive perks.
✅ 집에 손님이 오면 무서워서 짖는 강아지. 특히 남성에게 심하다고 합니다.(유튜브판 개는 훌륭하다) 저장 완료!
✅ 줄을 당기면 버티고 앉아서 산책 불가능. 어쩔 수 없이 안고 산책하신 보호자님. (유튜브판 개는 훌륭하다) 저장 완료!
✅ 7년 동안 아파트 단지 안에서만 산책한 강아지. 보호자님이 단지 밖으로 나가서 산책하는 게 소원이라고 하셨어요.(유튜브판 개는 훌륭하다) 저장 완료!


ERROR: [youtube] esxLzRxSnEE: This video is available to this channel's members on level: 댕밀리 (or any higher level). Join this channel to get access to members-only content and other exclusive perks.


⚠️ https://www.youtube.com/watch?v=esxLzRxSnEE 처리 실패: ERROR: [youtube] esxLzRxSnEE: This video is available to this channel's members on level: 댕밀리 (or any higher level). Join this channel to get access to members-only content and other exclusive perks.
⚠️ https://www.youtube.com/watch?v=g4oPzVpF41k 처리 실패: no element found: line 1, column 0
✅ 자동차에 타지 않으려고 버티는 강아지 훈련 방법 저장 완료!
⚠️ https://www.youtube.com/watch?v=_-TuWKkQ0mI 처리 실패: no element found: line 1, column 0
✅ 애견카페 “이건” 꼭 알고 가세요 저장 완료!
✅ 산책할 때 줄당김이 심한 강아지 훈련 방법과 반려인이라면 꼭 지켜야 하는 “펫티켓” 저장 완료!
✅ 청소기 돌리면 짖고 무는 강아지 훈련 방법 2 저장 완료!
⚠️ https://www.youtube.com/watch?v=OIajnt-FjV0 처리 실패: no element found: line 1, column 0
✅ 산책할 때 캥거루처럼 두 발로 걷는 강아지 훈련 방법 저장 완료!
✅ 어린 아이와 옆집 강아지 소리에 짖고 달려드는 강아지 훈련 방법 저장 완료!
✅ 엘리베이터에서 짖는 강아지 훈련 방법 저장 완료!
✅ 산책 할 때 짖는 반려견 대체 뭐가 문제일까? 저장 완료!
✅ 노즈워크 재미있게 해주는 방법 저장 완료!
✅ 단 한번의 교육훈련으로 보호자와 반려견이 변화 하는 과정 저장 완료!
✅ 어린 강아지 빗질 목을 잡고 하면 쉬워요 저장 완료!
✅ 짖는 강아지 어떻게 교육 할까? 저장 완료!
📄 모든 유튜브 데이터 저장 완료!
